In [46]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [47]:
import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
11.0
GCC 7.3


In [48]:
# 모듈 import
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [57]:
seed = 42

set_random_seed(seed, deterministic=False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [58]:
from mmcv import Config

# config file 들고오기
baseline_cfg_path = './configs/detectors/detectors_cascade_rcnn_Custom.py'
cfg = Config.fromfile(baseline_cfg_path)

In [59]:
model_name = 'detectors_cascade_rcnn_Custom.py'
exp = 'basic'

# Folder to store model logs and weight files
job_folder = f'./working/exp[{exp}]_{model_name}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'sunhyuk'
wnb_project_name = 'cv-3-bitcoin'

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: ./working/exp[basic]_detectors_cascade_rcnn_Custom.py


In [60]:
# dataset settings
dataset_type = 'CocoDataset'
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
root='../../dataset/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
print(cfg.pretty_text)

model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet101'),
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True),
    neck=dict(
        type='RFP',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        rfp_steps=2,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requir

In [61]:
albu_train_transforms=[
    dict(
        type='HorizontalFlip',
        p=0.5,
    ),
    dict(
        type='VerticalFlip',
        p=0.5,
    ),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.3),
    dict(
        type='HueSaturationValue',
        p=0.3,
    ),
    dict(
        dict(type="IAAAdditiveGaussianNoise"),
        p=0.3,
    ),
    dict(
        type='Sharpen',
        p=0.3,
    ),
    dict(
        type='RandomSizedBBoxSafeCrop',
        width=768,
        height=768,
        p=0.1
    )
]

# cfg.train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations', with_bbox=True),
#     dict(type='Resize', img_scale=(1024, 1024), keep_ratio=True),
#     dict(type='RandomFlip', flip_ratio=0.5),
#     # dict(
#     #     type='Albu',
#     #     transforms=albu_train_transforms,
#     #     bbox_params=dict(
#     #         type="BboxParams",
#     #         format="pascal_voc",
#     #         label_fields=["gt_labels"],
#     #         min_visibility=0.0,
#     #         filter_lost_elements=True,
#     #     ),
#     #     keymap={"img": "image", "gt_bboxes": "bboxes"},
#     #     update_pad_shape=False,
#     # ),
#     dict(type='Normalize', **img_norm_cfg),
#     dict(type='Pad', size_divisor=32),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
# ]
# cfg.test_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
#         img_scale=(1024, 1024),
#         flip=False,
#         transforms=[
#             dict(type='Resize', keep_ratio=True),
#             dict(type='RandomFlip'),
#             dict(type='Normalize', **img_norm_cfg),
#             dict(type='Pad', size_divisor=32),
#             dict(type='ImageToTensor', keys=['img']),
#             dict(type='Collect', keys=['img']),
#         ])
# ]
cfg.data = dict(
    imgs_per_gpu=8,#####################
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        ann_file= root + 'cv_train_1.json',
        img_prefix=root,
        pipeline=cfg.train_pipeline,
        classes = classes),
    val=dict(
        type=dataset_type,
        ann_file= root + 'cv_val_1.json',
        img_prefix=root,
        pipeline=cfg.test_pipeline,
        classes = classes),
    test=dict(
        type=dataset_type,
        ann_file= root + 'test.json',
        img_prefix=root,
        pipeline=cfg.test_pipeline,
        classes = classes),
)


In [62]:
cfg.seed = seed #~~~
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

# cfg.model.roi_head.bbox_head.num_classes = 10 ##클래수 수
# cfg.model.bbox_head.num_classes = 10

cfg.evaluation = dict(
    interval=1,
    save_best='auto',
    metric='bbox',
    classwise=True,
)

#
# cfg.optimizer = dict(type='SGD', lr=0.001, momentum=0.9, weight_decay=0.0001)
# cfg.optimizer = dict(type='Adam', lr=3e-4)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=50,
#     warmup_ratio=0.001,
#     step=[8, 11]
# )


# cfg.runner.max_epochs = 12

wandb_cfg = dict(config={"epochs": cfg.runner.max_epochs, "optimizer":cfg.optimizer, "model": cfg.model, "img_norm_cfg" : cfg.img_norm_cfg, "train_pipeline" : cfg.train_pipeline, "test_pipeline" : cfg.test_pipeline, "data" : cfg.data, "lr_config" : cfg.lr_config})

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         interval=10,
         init_kwargs=dict(config=wandb_cfg, project=wnb_username,entity=wnb_project_name, name=f'[{exp}]_{model_name}')
        )
]
cfg.log_config.interval = 10
# cfg.checkpoint_config = 20
print(cfg.pretty_text)


model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet101'),
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True),
    neck=dict(
        type='RFP',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        rfp_steps=2,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requir

In [63]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!



CocoDataset Train dataset with number of images 3677, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2993  | 1 [Paper]     | 4685  | 2 [Paper pack]  | 681   | 3 [Metal]   | 711   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2211  | 6 [Styrofoam] | 970   | 7 [Plastic bag] | 3808  | 8 [Battery] | 136   | 9 [Clothing] | 378   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [64]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-04-06 18:28:41,219 - mmdet - INFO - load checkpoint from torchvision path: torchvision://resnet101
2022-04-06 18:28:41,695 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset_l.weight, layer2.1.conv2.offset_l.bias, layer2.2.c

In [17]:
# 모델 학습
train_detector(model, datasets[0], cfg=cfg, distributed=False, validate=True)

2022-04-06 17:43:33,280 - mmdet - WARNING - "imgs_per_gpu" is deprecated in MMDet V2.0. Please use "samples_per_gpu" instead
2022-04-06 17:43:33,282 - mmdet - WARNING - Automatically set "samples_per_gpu"="imgs_per_gpu"=8 in this experiments
2022-04-06 17:43:36,519 - mmdet - INFO - Start running, host: root@642ef2a3bc92, work_dir: /opt/ml/detection/baseline/mmdetection/working/exp[basic]_detectors_htc_r101_20e_coco.py
2022-04-06 17:43:36,520 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook    

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/mymmdet/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/mymmdet/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/envs/mymmdet/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/mymmdet/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/ml/detection/baseline/mmdetection/mmdet/datasets/custom.py", line 218, in __getitem__
    data = self.prepare_train_img(idx)
  File "/opt/ml/detection/baseline/mmdetection/mmdet/datasets/custom.py", line 241, in prepare_train_img
    return self.pipeline(results)
  File "/opt/ml/detection/baseline/mmdetection/mmdet/datasets/pipelines/compose.py", line 41, in __call_